# agentscope.memory

agentscope.memory 模块是 AgentScope 框架中用于管理 智能体记忆（Memory） 的核心组件，分为两类： 
- 短期记忆（Short-term Memory）：对话历史，通常由 InMemoryMemory 实现。
- 长期记忆（Long-term Memory）：持久化、可检索的知识库，由 LongTermMemoryBase 及其实现（如 Mem0LongTermMemory）提供。
     

## 一、MemoryBase（抽象基类） 

所有记忆类的基类，定义了三个核心异步方法：
```python
async def add(*args, **kwargs)      # 添加记忆
async def delete(*args, **kwargs)   # 删除记忆
async def retrieve(*args, **kwargs) # 检索记忆
```

In [3]:
from agentscope.memory import InMemoryMemory
from agentscope.message import Msg

# 创建内存记忆
memory = InMemoryMemory()

# 添加消息
await memory.add(Msg("User", "你好！", "user"))
await memory.add(Msg("Agent", "你好！有什么我可以帮你的吗？", "assistant"))

# 使用正确的方法获取对话历史
history = await memory.get_memory()  # 这是一个 list[Msg]

print("对话历史:")
for msg in history:
    print(f"{msg.name}: {msg.content}")

对话历史:
User: 你好！
Agent: 你好！有什么我可以帮你的吗？


> 在 ReActAgent 中，若不传 memory 参数，会自动创建一个 InMemoryMemory 实例。

In [4]:
## 二、InMemoryMemory 详细示例

from agentscope.memory import InMemoryMemory
from agentscope.message import Msg

# 创建内存记忆实例
memory = InMemoryMemory()

# 添加多条消息
messages = [
    Msg("User", "我想了解狼人杀游戏", "user"),
    Msg("Assistant", "狼人杀是一个策略推理游戏...", "assistant"),
    Msg("User", "游戏有哪些角色？", "user"),
    Msg("Assistant", "主要角色包括：狼人、村民、预言家、女巫等", "assistant")
]

for msg in messages:
    await memory.add(msg)

# 检查内存大小
size = await memory.size()
print(f"当前内存中有 {size} 条消息")

# 获取所有对话历史
history = await memory.get_memory()
print("\n完整对话历史:")
for i, msg in enumerate(history):
    print(f"{i+1}. {msg.name}: {msg.content}")

# 删除指定索引的消息（删除第一条）
await memory.delete(0)
print(f"\n删除后内存大小: {await memory.size()}")

# 清空内存
await memory.clear()
print(f"清空后内存大小: {await memory.size()}")

当前内存中有 4 条消息

完整对话历史:
1. User: 我想了解狼人杀游戏
2. Assistant: 狼人杀是一个策略推理游戏...
3. User: 游戏有哪些角色？
4. Assistant: 主要角色包括：狼人、村民、预言家、女巫等

删除后内存大小: 3
清空后内存大小: 0


In [5]:
## 三、内存状态保存与加载

# 重新添加一些消息
await memory.add(Msg("User", "新的对话开始", "user"))
await memory.add(Msg("Assistant", "好的，我来帮助您", "assistant"))

# 保存内存状态
state = memory.state_dict()
print("内存状态:", state)

# 创建新的内存实例并加载状态
new_memory = InMemoryMemory()
new_memory.load_state_dict(state)

# 验证状态是否正确加载
restored_history = await new_memory.get_memory()
print("\n恢复的对话历史:")
for msg in restored_history:
    print(f"{msg.name}: {msg.content}")

内存状态: {'content': [{'id': 'Urhg3ctFEU6g2mg5Q4vNc5', 'name': 'User', 'role': 'user', 'content': '新的对话开始', 'metadata': None, 'timestamp': '2025-10-02 23:42:37.536'}, {'id': '2F33fgnc8Qz4Zh7nuCf63J', 'name': 'Assistant', 'role': 'assistant', 'content': '好的，我来帮助您', 'metadata': None, 'timestamp': '2025-10-02 23:42:37.536'}]}

恢复的对话历史:
User: 新的对话开始
Assistant: 好的，我来帮助您


In [6]:
## 四、防重复添加示例

# 创建具有相同ID的消息
msg1 = Msg("User", "重复消息测试", "user")
msg1.id = "test_id_001"

msg2 = Msg("User", "另一条重复消息", "user") 
msg2.id = "test_id_001"  # 相同的ID

# 默认不允许重复
await memory.clear()
await memory.add(msg1)
try:
    await memory.add(msg2)  # 这会被忽略，因为ID重复
except:
    pass

print(f"不允许重复时，内存大小: {await memory.size()}")

# 允许重复添加
await memory.add(msg2, allow_duplicates=True)
print(f"允许重复后，内存大小: {await memory.size()}")

不允许重复时，内存大小: 1
允许重复后，内存大小: 2


In [7]:
## 五、批量操作示例

# 批量添加消息
batch_messages = [
    Msg("Player1", "我是村民", "user"),
    Msg("Player2", "我怀疑Player3是狼人", "user"), 
    Msg("Player3", "我不是狼人！", "user"),
    Msg("Moderator", "请大家投票", "system")
]

await memory.clear()
await memory.add(batch_messages)  # 一次性添加多条消息

print(f"批量添加后内存大小: {await memory.size()}")

# 批量删除（删除多个索引）
await memory.delete([0, 2])  # 删除索引0和2的消息
print(f"批量删除后内存大小: {await memory.size()}")

remaining_history = await memory.get_memory()
print("\n剩余消息:")
for msg in remaining_history:
    print(f"{msg.name}: {msg.content}")

批量添加后内存大小: 4
批量删除后内存大小: 2

剩余消息:
Player2: 我怀疑Player3是狼人
Moderator: 请大家投票


In [9]:
## 六、在Agent中使用内存示例

from agentscope.agent import ReActAgent
from agentscope.model import DashScopeChatModel
from agentscope.formatter import DashScopeChatFormatter
from agentscope.memory import InMemoryMemory
from agentscope.message import Msg
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 创建自定义内存
custom_memory = InMemoryMemory()

# 预先添加一些背景信息
background_msgs = [
    Msg("System", "你是狼人杀游戏的主持人", "system"),
    Msg("System", "当前游戏有8名玩家参与", "system"),
    Msg("System", "游戏角色包括：4个村民、2个狼人、1个预言家、1个女巫", "system")
]

for msg in background_msgs:
    await custom_memory.add(msg)

# 创建狼人杀游戏主持人Agent
game_moderator = ReActAgent(
    name="GameModerator",
    sys_prompt="你是一个专业的狼人杀游戏主持人，负责主持游戏流程、宣布游戏结果和维持游戏秩序。",
    model=DashScopeChatModel(
        model_name="qwen-max",
        api_key=os.environ["DASHSCOPE_API_KEY"],
        stream=True,
        enable_thinking=True,
    ),
    formatter=DashScopeChatFormatter(),
    memory=custom_memory,  # 使用自定义内存
)

# 查看Agent的初始内存内容
print("Agent的初始内存:")
agent_memory = await custom_memory.get_memory()
for msg in agent_memory:
    print(f"{msg.name}: {msg.content}")

# 测试Agent功能
test_msg = Msg(
    name="player1",
    content="主持人，请开始游戏！",
    role="user"
)

print("\n=== 开始与狼人杀主持人对话 ===")
response = await game_moderator(test_msg)
print(f"主持人回复: {response.content}")

# 查看对话后的内存状态
print("\n=== 对话后的内存状态 ===")
updated_memory = await custom_memory.get_memory()
for i, msg in enumerate(updated_memory):
    print(f"{i+1}. {msg.name}: {msg.content}")

Agent的初始内存:
System: 你是狼人杀游戏的主持人
System: 当前游戏有8名玩家参与
System: 游戏角色包括：4个村民、2个狼人、1个预言家、1个女巫

=== 开始与狼人杀主持人对话 ===
GameModerator: 好的，欢迎各位玩家来到这场狼人杀游戏。在我们开始之前，请大家确认已经了解自己的身份，并且准备好进入神秘的夜晚。现在，请所有人闭上眼睛，我们将开始第一个夜晚。

【所有玩家请闭眼】
GameModerator: 夜晚降临，村民们沉睡了。狼人们，请睁开你们的眼睛，互相确认同伴并选择今晚要袭击的目标。请记住不要发出声音，通过眼神交流做出决定。选择完毕后，请再次闭上眼睛。
主持人回复: 夜晚降临，村民们沉睡了。狼人们，请睁开你们的眼睛，互相确认同伴并选择今晚要袭击的目标。请记住不要发出声音，通过眼神交流做出决定。选择完毕后，请再次闭上眼睛。

=== 对话后的内存状态 ===
1. System: 你是狼人杀游戏的主持人
2. System: 当前游戏有8名玩家参与
3. System: 游戏角色包括：4个村民、2个狼人、1个预言家、1个女巫
4. player1: 主持人，请开始游戏！
5. GameModerator: [{'type': 'text', 'text': '好的，欢迎各位玩家来到这场狼人杀游戏。在我们开始之前，请大家确认已经了解自己的身份，并且准备好进入神秘的夜晚。现在，请所有人闭上眼睛，我们将开始第一个夜晚。\n\n【所有玩家请闭眼】'}, {'type': 'tool_use', 'id': 'call_089951c9fd944aa1934a25', 'name': 'generate_response', 'input': {'response': '夜晚降临，村民们沉睡了。狼人们，请睁开你们的眼睛，互相确认同伴并选择今晚要袭击的目标。请记住不要发出声音，通过眼神交流做出决定。选择完毕后，请再次闭上眼睛。'}}]
6. system: [{'type': 'tool_result', 'id': 'call_089951c9fd944aa1934a25', 'name': 'generate_response', 'output': [{'type': 'text', 'text': 'Successf

In [10]:
## 七、内存检索和过滤示例

# 添加带有不同角色的消息
await memory.clear()
game_messages = [
    Msg("Moderator", "游戏开始，现在是第一天", "system"),
    Msg("Player1", "我是预言家，昨晚查验Player2是好人", "user"),
    Msg("Player2", "谢谢预言家验证", "user"),
    Msg("Player3", "我觉得Player1不像预言家", "user"),
    Msg("Moderator", "请大家开始讨论", "system"),
    Msg("Player4", "我投Player3", "user")
]

for msg in game_messages:
    await memory.add(msg)

# 虽然InMemoryMemory的retrieve方法为空实现，但我们可以手动过滤
all_messages = await memory.get_memory()

# 过滤出主持人的消息
moderator_messages = [msg for msg in all_messages if msg.name == "Moderator"]
print("主持人消息:")
for msg in moderator_messages:
    print(f"- {msg.content}")

# 过滤出玩家的发言
player_messages = [msg for msg in all_messages if msg.name.startswith("Player")]
print(f"\n玩家发言 ({len(player_messages)} 条):")
for msg in player_messages:
    print(f"- {msg.name}: {msg.content}")

主持人消息:
- 游戏开始，现在是第一天
- 请大家开始讨论

玩家发言 (4 条):
- Player1: 我是预言家，昨晚查验Player2是好人
- Player2: 谢谢预言家验证
- Player3: 我觉得Player1不像预言家
- Player4: 我投Player3
